# Grundlagen

## Design

Bei Auswahl von Farben, Schriften und Ähnlichem orientiere ich mich am Desgin des Bundesrats. Mit meinem Projekt möchte ich den Bundesrat dahingehend beraten, ob der Gesetzesentwurf fair ist oder nicht.

<img src="C:/data_storytelling/books/datast_project/data/2_bundesrat.png" alt="Screenshot der Bundesrat-Website" width="50%"/>

Quelle: [Bundesrat-Website](https://www.bundesrat.de/DE/homepage/homepage-node.html)

Ich nutze also Serifenschrift. Die Farben sind vorwiegend Grau und Dunkelrot. Weil ich noch eine zweite Farbe brauche, habe ich einen etwas helleren Rotton hinzugefügt.

## Daten

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/1_dbs_data.csv', delimiter=';', header=[2,3], encoding='ISO-8859-1', low_memory=False)

sum_row = df.loc[(df[('NR','Unnamed: 0_level_1')] == 'Summe')]

library_df = sum_row.astype('string')

for label, content in library_df.iteritems():
    library_df[label] = library_df[label].str.replace('.', '', regex=True)
    library_df[label] = library_df[label].str.replace(',', '.', regex=True)

library_df = pd.melt(library_df)
library_df = library_df.rename(columns={"variable_0": "Auspraegung", 'variable_1': 'Jahr', 'value': 'Wert'})

library_df = library_df.drop([0, 1, 2, 3, 103])

library_df = library_df.pivot(index='Jahr', columns='Auspraegung', values='Wert')

for label, content in library_df.iteritems():
    library_df[label] = library_df[label].astype(float)

library_df.reset_index(inplace=True)

pop_df = pd.read_csv('./data/1_pop_data.csv', delimiter=';', encoding='ISO-8859-1', header=[5])

pop_df = pop_df.drop([11, 12, 13, 14])

pop_df = pop_df.rename(columns={"Unnamed: 0": "Jahr"})

pop_df['Anzahl'] = pop_df['Anzahl'].astype(int)

pop_df['Jahr'] = pd.to_datetime(pop_df['Jahr'], format='%d.%m.%Y')

df = pd.DataFrame(
    {
        # general info
        'year' : ([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]),
        'population' : pop_df['Anzahl'],
        
        # reading habits
        'digital_readers': ([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 7650000, 8520000, 8680000, 9120000]),

        'readers_weekly' : ([20400000, 20500000, 20000000, 19300000, 18500000, 18100000, 18100000, 11640000, 11930000, 11960000, 12620000]),
        'readers_monthly' : ([20900000, 20500000, 20600000, 21100000, 21300000, 20200000, 20100000, 15070000, 14840000, 14730000, 14710000]),
        'readers_once_month' : ([12000000, 11500000, 10100000, 10000000, 10400000, 10400000, 10100000, 6870000, 7060000, 7110000, 7030000]),

        # publishing information
        'ebook_sales': ([1900000, 4300000, 13200000, 21500000, 24800000, 27000000, 28100000, 29100000, 32800000, 32400000, 35800000]),

        # library information
        'lenders': library_df['Entleiher'],
        'lendings': library_df['Entleih. insges.'],
        'digital_lendings': library_df['Entl. virt.Best.'], 
        
    }
)

df['readers'] = df['readers_weekly'] + df['readers_monthly'] + df['readers_once_month']

# readers relative to the population over the years
df['perc_digital_readers'] = df['digital_readers'] / df['population']
df['perc_readers'] = df['readers'] / df['population']

## Visualisierung

In [4]:
# import modules
import plotly.express as px
import plotly.graph_objects as go

### Lesegewohnheiten der Deutschen

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['year'], y=df['perc_readers'], name='lesen regelmäßig', line=dict(color='#CC9DA8', width=3)))
fig.add_trace(go.Scatter(x=df['year'], y=df['perc_digital_readers'], name='lesen E-Books', line=dict(color='#810a26', width=3)))

fig.add_vrect(x0='2020-03-01', x1='2021-01-25', fillcolor='#e2e2e2', opacity=0.2, line=dict(color='#555555', width=1))

fig.update_layout(title='Lesegewohnheiten der Deutschen',
                    title_font_size=25,

                    xaxis_title='Jahr',
                    yaxis_title='Anteil',

                    font_family='Gravitas One',
                    font_color='#555555',

                    paper_bgcolor='#e2e2e2',
                    plot_bgcolor='#ffffff',
                   )

fig.add_hline(df['perc_digital_readers'].iloc[-1], line=dict(color='#555555', width=1, dash='dash'))

fig.update_xaxes(title_text='Jahr', ticks='inside', tickcolor='#555555', fixedrange=True)
fig.update_yaxes(dtick=0.2, tick0=0, tickformat="%", ticks='inside', fixedrange=True, range=[0, 1], tickcolor='#555555')

fig.show()

### Ausleih- und Verkaufszahlen von E-Books

In [12]:
fig = go.Figure()

# traces: lendings and sales
fig.add_trace(go.Scatter(x=df['year'], y=df['digital_lendings'], name='Ausleihen', line=dict(color='#CC9DA8', width=3)))
fig.add_trace(go.Scatter(x=df['year'], y=df['ebook_sales'], name='Verkäufe', line=dict(color='#810a26', width=3)))

fig.update_traces(hovertemplate='%{x}: %{y} E-Books')

# layout
fig.update_layout(title='Ausleih- und Verkaufszahlen von E-Books',
                    title_font_size=25,

                    xaxis_title='Jahr',
                    yaxis_title='Stückzahl',

                    font_family='Gravitas One',
                    font_color='#555555',

                    paper_bgcolor='#e2e2e2',
                    plot_bgcolor='#ffffff',
                   )

fig.update_xaxes(title_text='Jahr', ticks='inside', tickcolor='#555555', fixedrange=True)
fig.update_yaxes(dtick=10000000, tick0=0, tickformat='.2s', ticks='inside', fixedrange=True, range=[0, 45000000], tickcolor='#555555')

# adding markers to indicate covid-19 and the point where lendings become higher than sales
fig.add_vline('2019', line=dict(color='#555555', width=2, dash='dash'))
fig.add_vrect(x0='2019.5', x1='2021', fillcolor='#e2e2e2', opacity=0.2, line=dict(color='#555555', width=1))

fig.show()